# Scrapping

In [2]:
from os import WIFSIGNALED
import requests
import pandas as pd
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter



# Testing

In [ ]:
city_name = "barcelone"
api_key = "" ## Put your key api here :)))
date = "2025-11-05"

def get_weather(api_key, city, date):
    geolocator = Nominatim(user_agent="myapp")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)
    location = geocode(city)
    if location is None:
        print(f"Location not found for city: {city}")
        return {
            'Match_date': date,
            'City': city,
            'Temperature': None,
            'Feels_like': None,
            'Humidity': None,
            'Wind_speed': None,
            'Precipitation': None,
            'Clouds': None
        }

    lat = location.latitude
    lon = location.longitude
    url = f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={lat}&lon={lon}&date={date}&appid={api_key}"

    response = requests.get(url).json()
    # print(response)

    temp = response['temperature']['afternoon']
    temp = math.floor((temp * 1.8) - 459.67)

    feels_like = response['temperature']['afternoon']
    feels_like = math.floor((feels_like * 1.8) - 459.67)

    humidity = response['humidity']['afternoon']

    wind_speed = response['wind']['max']['speed']

    precipitation = response['precipitation']['total']

    clouds = response['cloud_cover']['afternoon']

    return {
        'Match_date': date,
        'City': city,
        'Temperature': temp,
        'Feels_like': feels_like,
        'Humidity': humidity,
        'Wind_speed': wind_speed,
        'Precipitation': precipitation,
        'Clouds': clouds
    }

weather = get_weather(api_key, city_name, date)



In [4]:
weather_df = pd.DataFrame([weather])
weather_df.head()

,Match_date,City,Temperature,Feels_like,Humidity,Wind_speed,Precipitation,Clouds
0,2025-11-05,barcelone,61,61,66.1,4.93,0.07,57.33


# Our use case

## extract date of matches from the table of matches and then extract the weathers infos for each game

In [5]:
# path to change
matches = pd.read_csv('/content/match_barcelone.csv')
matches.head()

,Match_date,Time,Opponent,Competition,Round,Day,Venue,Result,Goals_for,Goals_against,Captain_firstname,Captain_lastname,Possession,Formation,Opponent_formation,Attendance,Referee
0,13/08/2023,21:30,Getafe,La Liga,Matchweek 1,Sun,Away,D,0,0,Marc-André,ter Stegen,74,4-2-3-1,4-5-1,"13,410",César Soto
1,20/08/2023,19:30,Cádiz,La Liga,Matchweek 2,Sun,Home,W,2,0,Marc-André,ter Stegen,74,3-4-3,4-4-2,"39,603",Alejandro Muñíz
2,27/08/2023,17:30,Villarreal,La Liga,Matchweek 3,Sun,Away,W,4,3,Sergi,Roberto,60,4-3-3,4-4-2,"21,679",Alejandro Hernández
3,03/09/2023,21:00,Osasuna,La Liga,Matchweek 4,Sun,Away,W,2,1,Sergi,Roberto,69,4-3-3,4-1-4-1,"21,966",Miguel Ángel Ortiz Arias
4,16/09/2023,21:00,Betis,La Liga,Matchweek 5,Sat,Home,W,5,0,Marc-André,ter Stegen,65,4-3-3,4-2-3-1,"45,055",José Sánchez


In [6]:
matches['Match_date'] = pd.to_datetime(matches['Match_date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

In [7]:
matches.head()

,Match_date,Time,Opponent,Competition,Round,Day,Venue,Result,Goals_for,Goals_against,Captain_firstname,Captain_lastname,Possession,Formation,Opponent_formation,Attendance,Referee
0,2023-08-13,21:30,Getafe,La Liga,Matchweek 1,Sun,Away,D,0,0,Marc-André,ter Stegen,74,4-2-3-1,4-5-1,"13,410",César Soto
1,2023-08-20,19:30,Cádiz,La Liga,Matchweek 2,Sun,Home,W,2,0,Marc-André,ter Stegen,74,3-4-3,4-4-2,"39,603",Alejandro Muñíz
2,2023-08-27,17:30,Villarreal,La Liga,Matchweek 3,Sun,Away,W,4,3,Sergi,Roberto,60,4-3-3,4-4-2,"21,679",Alejandro Hernández
3,2023-09-03,21:00,Osasuna,La Liga,Matchweek 4,Sun,Away,W,2,1,Sergi,Roberto,69,4-3-3,4-1-4-1,"21,966",Miguel Ángel Ortiz Arias
4,2023-09-16,21:00,Betis,La Liga,Matchweek 5,Sat,Home,W,5,0,Marc-André,ter Stegen,65,4-3-3,4-2-3-1,"45,055",José Sánchez


In [8]:
weather_data = []

for index, row in matches.iterrows():
    date = row['Match_date']
    venue = row['Venue']

    if venue == 'Away':
        team_location = row['Opponent']
    else:
        team_location = "Barcelone"

    weather_info = get_weather(api_key, team_location, date)
    weather_data.append(weather_info)

weather_df = pd.DataFrame(weather_data)

Location not found for city: Unionistas Sal


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/ssl.py", line 1159, in read
    return self._sslobj.read(len, buffer)
TimeoutError: The read operation timed out

The above e

Location not found for city: it Napoli


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/ssl.py", line 1159, in read
    return self._sslobj.read(len, buffer)
TimeoutError: The read operation timed out

The above e

Location not found for city: fr Paris S-G


In [9]:
weather_df.head(10)

,Match_date,City,Temperature,Feels_like,Humidity,Wind_speed,Precipitation,Clouds
0,2023-08-13,Getafe,84.0,84.0,48.0,7.20,0.00,0.0
1,2023-08-20,Barcelone,92.0,92.0,61.0,6.17,0.00,0.0
2,2023-08-27,Villarreal,79.0,79.0,43.0,11.33,2.38,93.0
3,2023-09-03,Osasuna,37.0,37.0,51.0,9.18,0.00,7.0
4,2023-09-16,Barcelone,78.0,78.0,78.0,6.17,0.67,20.0
5,2023-09-19,Barcelone,78.0,78.0,70.0,6.17,0.00,20.0
6,2023-09-23,Barcelone,72.0,72.0,56.0,6.69,0.00,20.0
7,2023-09-26,Mallorca,78.0,78.0,52.0,5.14,0.00,20.0
8,2023-09-29,Barcelone,79.0,79.0,75.0,4.12,0.00,0.0
9,2023-10-04,pt Porto,72.0,72.0,70.0,3.60,0.00,0.0


## Changing date format from yyyy-mm-dd to dd/mm/yyyy

In [12]:
weather_df['Match_date'] = pd.to_datetime(weather_df['Match_date']).dt.strftime("%d/%m/%Y")

In [13]:
weather_df.head()

,Match_date,City,Temperature,Feels_like,Humidity,Wind_speed,Precipitation,Clouds
0,13/08/2023,Getafe,84.0,84.0,48.0,7.20,0.00,0.0
1,20/08/2023,Barcelone,92.0,92.0,61.0,6.17,0.00,0.0
2,27/08/2023,Villarreal,79.0,79.0,43.0,11.33,2.38,93.0
3,03/09/2023,Osasuna,37.0,37.0,51.0,9.18,0.00,7.0
4,16/09/2023,Barcelone,78.0,78.0,78.0,6.17,0.67,20.0


### change temp from fahrenheit to celsius

In [14]:
def fahrenheit_to_celsius(f):
    return (f - 32) / 1.8

weather_df['Temperature'] = weather_df['Temperature'].apply(fahrenheit_to_celsius)
weather_df.head()

,Match_date,City,Temperature,Feels_like,Humidity,Wind_speed,Precipitation,Clouds
0,13/08/2023,Getafe,28.888889,84.0,48.0,7.20,0.00,0.0
1,20/08/2023,Barcelone,33.333333,92.0,61.0,6.17,0.00,0.0
2,27/08/2023,Villarreal,26.111111,79.0,43.0,11.33,2.38,93.0
3,03/09/2023,Osasuna,2.777778,37.0,51.0,9.18,0.00,7.0
4,16/09/2023,Barcelone,25.555556,78.0,78.0,6.17,0.67,20.0


## Save

In [15]:
weather_df.to_csv('weather.csv', index=False)